In [1]:
import pandas as pd

# Step 1: Load the CSV without the "Unnamed: 0" column
combined_df = pd.read_csv('All_in_one.csv', index_col=0)

# Step 2: Save it without adding an index
# combined_df.to_csv('All_in_one.csv', index=False)

In [11]:
combined_df = pd.read_csv('All_in_one.csv')
# combined_df = combined_df.loc[:, ~combined_df.columns.str.contains('^Unnamed')]
# combined_df['date'] = pd.to_datetime(combined_df['date'])
# combined_df.to_csv('All_in_one.csv', index=False)
combined_df

,date,message,channel_name,city
0,2024-10-31 19:19:02,Ищем на подселение 1 девушку Квартира 3 комнат...,SDU [housing],Almaty
1,2024-10-31 18:32:46,Всем привет ищу 12 девочку к себе на подселени...,SDU [housing],Almaty
2,2024-10-31 18:23:14,Алма Сити 5 екы жыгыт ыздеймыз поселениеге 50к...,SDU [housing],Almaty
3,2024-10-31 17:43:32,СДУға жақын жерде орналасқан коттедж үй жалға ...,SDU [housing],Almaty
4,2024-10-31 16:42:10,ищем одну девушку на подселение в однокомнатну...,SDU [housing],Almaty
...,...,...,...,...
71566,2024-09-19 11:59:23,Сниму квартиру в районе сахзаводана длительный...,ТАРАЗ КВАРТИРА ВРЕМЯНКА ДОМ СДАМ СНИМУ ПОДСЕЛЕНИЕ,Taraz
71567,2024-09-19 11:45:18,Подселение в 2х комнатную кв Проживание с ХОЗЯ...,ТАРАЗ КВАРТИРА ВРЕМЯНКА ДОМ СДАМ СНИМУ ПОДСЕЛЕНИЕ,Taraz
71568,2024-09-08 22:21:26,Здравствуйтепомогу с поиском квартиры по вашем...,ТАРАЗ КВАРТИРА ВРЕМЯНКА ДОМ СДАМ СНИМУ ПОДСЕЛЕНИЕ,Taraz
71569,2024-09-04 21:54:05,Сниму 12комнкв в 15 мкр на долгий срок,ТАРАЗ КВАРТИРА ВРЕМЯНКА ДОМ СДАМ СНИМУ ПОДСЕЛЕНИЕ,Taraz


In [12]:
len(combined_df)

71571

In [3]:
from gemini1 import gemini
import re

mes = combined_df['message'].iloc[1000]
print(mes)
print(gemini(mes))

Квартирага 1бала керек Ішпейтіншекпейтінтазалықты сақтайтын болу керек Квартира сдудан 5мин жерде 4бала болып турамыз Бір айға 500 00 Коммуналка бәрі ішінде Толығырақ жекеге
address: Квартира сдудан 5мин жерде
cost: 50000
gender: null
people_needed: 1
long_term: False
phone_number: null
utilities: True
deposit: null


In [41]:
new_df = (
    combined_df.sort_values(by=['channel_name', 'date'], ascending=[True, False])
    .groupby('channel_name')
    .head(10)
)
new_df['district'] = None
new_df['address'] = None
new_df['cost'] = None
new_df['gender'] = None
new_df['people_needed'] = None
new_df['long_term'] = None
new_df['phone_number'] = None
new_df['utilities'] = None
new_df['deposit'] = None

new_df

,date,message,channel_name,city,district,address,cost,gender,people_needed,long_term,phone_number,utilities,deposit
46520,2024-10-28 16:40:34,Привет ищем 3х девушек для подселения 3х комна...,DAILY ХАТА,Astana,None,None,None,None,None,None,None,None,None
46521,2024-10-27 17:30:22,Ищем третью девочку на подселение чистоплотную...,DAILY ХАТА,Astana,None,None,None,None,None,None,None,None,None
46522,2024-10-27 17:30:08,СРОЧНО Ищем одну девочку на заселение на долги...,DAILY ХАТА,Astana,None,None,None,None,None,None,None,None,None
46523,2024-10-27 17:29:54,СРОЧНО Ищем девочку на подселение в ЖК Будем ж...,DAILY ХАТА,Astana,None,None,None,None,None,None,None,None,None
46524,2024-10-25 17:00:55,Срочно ищем соседку Жк Ищем одну девочку на за...,DAILY ХАТА,Astana,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67956,2024-10-28 23:28:37,Келесі айдың 5і нен бастап көшетін кв керек 2 ...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,None,None,None,None
67957,2024-10-28 02:25:25,Квартира іздеймін 1 бөлмелі Барлық жағдайы жас...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,None,None,None,None
67958,2024-10-28 01:39:27,Ищу квартиру В районе пединститута либо школьн...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,None,None,None,None
67959,2024-10-20 21:44:37,Бирге туруга кв керек подселение Центр жактан ...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,None,None,None,None


In [43]:
for index, row in new_df.iterrows():
    message = row['message']
    data = gemini(message)  # Get the structured data as a string from gemini

    # Extract each value using regex and store in the appropriate column
    address = re.search(r'address: (.+)', data)
    cost = re.search(r'cost: (\d+)', data)
    gender = re.search(r'gender: (\w+)', data)
    people_needed = re.search(r'people_needed: (\d+)', data)
    long_term = re.search(r'long_term: (\w+)', data)
    phone_number = re.search(r'phone_number: (\S+)', data)
    utilities = re.search(r'utilities: (\w+)', data)
    deposit = re.search(r'deposit: (\d+)', data)

    # Update the DataFrame with extracted values (convert as needed)
    new_df.at[index, 'address'] = address.group(1) if address else None
    new_df.at[index, 'cost'] = int(cost.group(1)) if cost else None
    new_df.at[index, 'gender'] = gender.group(1) if gender else None
    new_df.at[index, 'people_needed'] = int(people_needed.group(1)) if people_needed else None
    new_df.at[index, 'long_term'] = True if long_term and long_term.group(1) == 'True' else False
    new_df.at[index, 'phone_number'] = phone_number.group(1) if phone_number else None
    new_df.at[index, 'utilities'] = True if utilities and utilities.group(1) == 'True' else False
    new_df.at[index, 'deposit'] = int(deposit.group(1)) if deposit else None
new_df

,date,message,channel_name,city,district,address,cost,gender,people_needed,long_term,phone_number,utilities,deposit
46520,2024-10-28 16:40:34,Привет ищем 3х девушек для подселения 3х комна...,DAILY ХАТА,Astana,None,null,60,female,3,False,null,True,None
46521,2024-10-27 17:30:22,Ищем третью девочку на подселение чистоплотную...,DAILY ХАТА,Astana,None,Жк Сенім,60,female,1,False,null,True,None
46522,2024-10-27 17:30:08,СРОЧНО Ищем одну девочку на заселение на долги...,DAILY ХАТА,Astana,None,ЖК Будем жить,60000,female,1,True,null,True,None
46523,2024-10-27 17:29:54,СРОЧНО Ищем девочку на подселение в ЖК Будем ж...,DAILY ХАТА,Astana,None,ЖК Будем жить,60,female,1,False,null,True,None
46524,2024-10-25 17:00:55,Срочно ищем соседку Жк Ищем одну девочку на за...,DAILY ХАТА,Astana,None,Жк Ищем одну девочку,65,female,1,False,null,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67956,2024-10-28 23:28:37,Келесі айдың 5і нен бастап көшетін кв керек 2 ...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,False,None,False,None
67957,2024-10-28 02:25:25,Квартира іздеймін 1 бөлмелі Барлық жағдайы жас...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,False,None,False,None
67958,2024-10-28 01:39:27,Ищу квартиру В районе пединститута либо школьн...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,null,2535,null,None,False,77788077747,False,None
67959,2024-10-20 21:44:37,Бирге туруга кв керек подселение Центр жактан ...,Уральск Квартира Орал КВАРТИРА Аренда,Oral,None,None,None,None,None,False,None,False,None


In [44]:
new_df.to_csv('result.csv', index=False)

In [45]:
new_df.to_excel('result.xlsx', index=False)

In [46]:
import time

# Select the last 100 messages by date for each channel
top_100_per_channel = (
    combined_df.sort_values(by=['channel_name', 'date'], ascending=[True, False])
    .groupby('channel_name')
    .head(100)
)

# Initialize columns for structured data fields
top_100_per_channel['address'] = None
top_100_per_channel['cost'] = None
top_100_per_channel['gender'] = None
top_100_per_channel['people_needed'] = None
top_100_per_channel['long_term'] = None
top_100_per_channel['phone_number'] = None
top_100_per_channel['utilities'] = None
top_100_per_channel['deposit'] = None

# Process each channel's data separately
for channel_name, channel_data in top_100_per_channel.groupby('channel_name'):
    print(f"Processing channel: {channel_name}")
    
    # Process in batches of 10
    for i in range(0, len(channel_data), 10):
        batch = channel_data.iloc[i:i+10]  # Select the batch of 10

        for index, row in batch.iterrows():
            message = row['message']
            data = gemini(message)  # Get structured data as a string from gemini

            # Extract each value using regex and store in the appropriate column
            address = re.search(r'address: (.+)', data)
            cost = re.search(r'cost: (\d+)', data)
            gender = re.search(r'gender: (\w+)', data)
            people_needed = re.search(r'people_needed: (\d+)', data)
            long_term = re.search(r'long_term: (\w+)', data)
            phone_number = re.search(r'phone_number: (\S+)', data)
            utilities = re.search(r'utilities: (\w+)', data)
            deposit = re.search(r'deposit: (\d+)', data)

            # Update the DataFrame with extracted values
            top_100_per_channel.at[index, 'address'] = address.group(1) if address else None
            top_100_per_channel.at[index, 'cost'] = int(cost.group(1)) if cost else None
            top_100_per_channel.at[index, 'gender'] = gender.group(1) if gender else None
            top_100_per_channel.at[index, 'people_needed'] = int(people_needed.group(1)) if people_needed else None
            top_100_per_channel.at[index, 'long_term'] = True if long_term and long_term.group(1) == 'True' else False
            top_100_per_channel.at[index, 'phone_number'] = phone_number.group(1) if phone_number else None
            top_100_per_channel.at[index, 'utilities'] = True if utilities and utilities.group(1) == 'True' else False
            top_100_per_channel.at[index, 'deposit'] = int(deposit.group(1)) if deposit else None

        # Wait 2 seconds after processing each batch of 10
        time.sleep(2)

# Save the processed data to a new CSV file
top_100_per_channel.to_csv('processed_top_100_per_channel.csv', index=False)
print("Processing complete. Data saved to 'processed_top_100_per_channel.csv'.")


Processing channel: DAILY ХАТА
Processing channel: SDU [housing]
Processing channel: podselenie_v_astane
Processing channel: АЛМАТЫ (КВАРЕНДАПОДСЕЛ🏠)
Processing channel: Аренда Жилья.Подселение Алматы
Processing channel: Аренда квартир Алматы - Чат
Processing channel: Аренда квартира Алматы
Processing channel: Аренда комнат и квартир Талдыкорган🏙🌃🌌🏢🏘
Processing channel: КВАРТИРА АЛМАТЫ  ПӘТЕР АЛМАТЫ  НЕДВИЖИМОСТЬ АЛМАТЫ
Processing channel: Квартира в Алматы  Поиск квартиры
Processing channel: Квартира. Сдам, Сниму г.Атырау
Processing channel: Квартиры Алматы  Аренда квартир Алматы  Поиск соседей Алматы
Processing channel: Квартиры Астана  Аренда квартир Астана  Поиск соседей Астана
Processing channel: ПОДСЕЛЕНИЕ АЛМАТЫ
Processing channel: ПОДСЕЛЕНИЕ АЛМАТЫ КВАРТИРА
Processing channel: ПОДСЕЛЕНИЕ АСТАНА КВАРТИРА
Processing channel: ПОДСЕЛЕНИЕ ШЫМКЕНТ АРЕНДА КВАРТИРА
Processing channel: Подселение Астана сдам сниму комнату квартиру Астана


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [48]:
top_100_per_channel.to_csv('processed_top_100_per_channel.csv', index=False)

In [49]:
top_100_per_channel.to_excel('processed_top_100_per_channel.xlsx', index=False)

In [ ]:
all_df = combined_df.copy()
all_df = all_df.sort_values(by='date', ascending=False)

# Initialize columns for structured data fields
all_df['address'] = None
all_df['cost'] = None
all_df['gender'] = None
all_df['people_needed'] = None
all_df['long_term'] = None
all_df['phone_number'] = None
all_df['utilities'] = None
all_df['deposit'] = None

# Process in batches of 10 for all data
for i in range(0, len(all_df), 10):
    batch = all_df.iloc[i:i+10]  # Select the batch of 10 messages

    # Process each message in the batch
    for index, row in batch.iterrows():
        message = row['message']
        data = gemini(message)  # Get the structured data as a string from gemini

        # Extract each value using regex and store in the appropriate column
        address = re.search(r'address: (.+)', data)
        cost = re.search(r'cost: (\d+)', data)
        gender = re.search(r'gender: (\w+)', data)
        people_needed = re.search(r'people_needed: (\d+)', data)
        long_term = re.search(r'long_term: (\w+)', data)
        phone_number = re.search(r'phone_number: (\S+)', data)
        utilities = re.search(r'utilities: (\w+)', data)
        deposit = re.search(r'deposit: (\d+)', data)

        # Update the DataFrame with extracted values
        all_df.at[index, 'address'] = address.group(1) if address else None
        all_df.at[index, 'cost'] = int(cost.group(1)) if cost else None
        all_df.at[index, 'gender'] = gender.group(1) if gender else None
        all_df.at[index, 'people_needed'] = int(people_needed.group(1)) if people_needed else None
        all_df.at[index, 'long_term'] = True if long_term and long_term.group(1) == 'True' else False
        all_df.at[index, 'phone_number'] = phone_number.group(1) if phone_number else None
        all_df.at[index, 'utilities'] = True if utilities and utilities.group(1) == 'True' else False
        all_df.at[index, 'deposit'] = int(deposit.group(1)) if deposit else None

    # Wait 2 seconds after processing each batch of 10
    time.sleep(2)

# Save the processed data to a new CSV file
all_df.to_csv('processed_all_data.csv', index=False)
all_df.to_excel('processed_all_data.xlsx', index=False)
print("Processing complete. Data saved to 'processed_all_data.csv'.")

In [52]:
all_df.to_excel('processed_all_data.xlsx', index=False)
